In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler

# ------------------------------
# 1. Data Loading and Preprocessing
# ------------------------------
def load_and_preprocess_ecg_data(csv_path="data/lead_I_data.csv"):
    """
    Loads and preprocesses ECG data from CSV, skipping the header row.
    
    Expected CSV:
      - First row: header (skipped)
      - Remaining rows: ~987 samples
      - 5000 columns: time steps
      
    Processing:
      1. Load CSV and skip the first row.
      2. Convert to a NumPy array.
      3. Scale values to [0,1] using MinMaxScaler.
      4. Reshape to [samples, timesteps, 1] (one ECG channel).
    
    Returns:
        data_scaled: np.array of shape (num_samples, 5000, 1)
        scaler: fitted MinMaxScaler (for later inversion if needed)
    """
    # Skip the header row with skiprows=1
    df = pd.read_csv(csv_path, header=None, skiprows=1)
    data = df.values  # Expected shape: (987, 5000)
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    data_scaled = scaler.fit_transform(data)
    
    # Reshape to [samples, timesteps, features]
    data_scaled = data_scaled.reshape(data_scaled.shape[0], data_scaled.shape[1], 1)
    
    return data_scaled, scaler

# Load the ECG data
ecg_data_np, ecg_scaler = load_and_preprocess_ecg_data()
print("ECG data shape:", ecg_data_np.shape)  # Expected: (987, 5000, 1)

# Convert to torch.Tensor
ecg_data_tensor = torch.tensor(ecg_data_np, dtype=torch.float32)

# Create a DataLoader
batch_size = 32
dataset = TensorDataset(ecg_data_tensor)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# ------------------------------
# 2. Define the Networks (Generator, Recovery, Discriminator)
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generator: maps noise (z) to a latent representation
class Generator(nn.Module):
    def __init__(self, z_dim, hidden_dim, num_layers, seq_len):
        """
        Args:
            z_dim (int): Dimensionality of the input noise.
            hidden_dim (int): Latent space dimension.
            num_layers (int): Number of GRU layers.
            seq_len (int): Length of the time series.
        """
        super(Generator, self).__init__()
        self.seq_len = seq_len
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.gru = nn.GRU(input_size=z_dim,
                          hidden_size=hidden_dim,
                          num_layers=num_layers,
                          batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim)
        self.tanh = nn.Tanh()
        
    def forward(self, z):
        out, _ = self.gru(z)  # (batch_size, seq_len, hidden_dim)
        out = self.fc(out)
        out = self.tanh(out)
        return out

# Recovery: maps the latent representation to ECG data
class Recovery(nn.Module):
    def __init__(self, hidden_dim, num_layers, seq_len, output_dim=1):
        """
        Args:
            hidden_dim (int): Latent space dimension.
            num_layers (int): Number of GRU layers.
            seq_len (int): Length of the time series.
            output_dim (int): Dimension of the output signal (1 for ECG).
        """
        super(Recovery, self).__init__()
        self.gru = nn.GRU(input_size=hidden_dim,
                          hidden_size=hidden_dim,
                          num_layers=num_layers,
                          batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.tanh = nn.Tanh()  # Assumes data scaled to [-1,1] or [0,1]
        
    def forward(self, h):
        out, _ = self.gru(h)  # (batch_size, seq_len, hidden_dim)
        out = self.fc(out)    # (batch_size, seq_len, output_dim)
        out = self.tanh(out)
        return out

# Discriminator: distinguishes real ECG from fake ECG
class Discriminator(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=24, num_layers=3, seq_len=5000):
        """
        Args:
            input_dim (int): Dimension of input signal (1 for ECG).
            hidden_dim (int): Hidden dimension for the GRU.
            num_layers (int): Number of GRU layers.
            seq_len (int): Length of the time series.
        """
        super(Discriminator, self).__init__()
        self.gru = nn.GRU(input_size=input_dim,
                          hidden_size=hidden_dim,
                          num_layers=num_layers,
                          batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        out, _ = self.gru(x)  # (batch_size, seq_len, hidden_dim)
        last_out = out[:, -1, :]  # Use the last time step as summary
        out = self.fc(last_out)
        out = self.sigmoid(out)
        return out

# Hyperparameters for networks
seq_len = 5000
z_dim = 24
hidden_dim = 24
num_layers = 3

# Instantiate networks and move to device
G = Generator(z_dim=z_dim, hidden_dim=hidden_dim, num_layers=num_layers, seq_len=seq_len).to(device)
R = Recovery(hidden_dim=hidden_dim, num_layers=num_layers, seq_len=seq_len, output_dim=1).to(device)
D = Discriminator(input_dim=1, hidden_dim=hidden_dim, num_layers=num_layers, seq_len=seq_len).to(device)

# ------------------------------
# 3. Define Optimizers and Loss Function
# ------------------------------
lr = 0.0002
optimizer_G = optim.Adam(list(G.parameters()) + list(R.parameters()), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))
criterion = nn.BCELoss()

In [ ]:
# ------------------------------
# 4. Full Training Loop (50 epochs, plotting every 5 epochs)
# ------------------------------
num_epochs = 20

for epoch in range(1, num_epochs + 1):
    for batch in data_loader:
        real_ecg = batch[0].to(device)  # Shape: (batch_size, seq_len, 1)
        current_bs = real_ecg.size(0)
        
        # Labels for real and fake data
        real_labels = torch.ones(current_bs, 1).to(device)
        fake_labels = torch.zeros(current_bs, 1).to(device)
        
        # --- Train Discriminator ---
        optimizer_D.zero_grad()
        
        # Discriminator loss on real data
        outputs_real = D(real_ecg)
        loss_D_real = criterion(outputs_real, real_labels)
        
        # Generate fake ECG:
        noise = torch.randn(current_bs, seq_len, z_dim).to(device)
        fake_latent = G(noise)
        fake_ecg = R(fake_latent)
        
        outputs_fake = D(fake_ecg.detach())  # Detach to avoid gradient flow into G & R
        loss_D_fake = criterion(outputs_fake, fake_labels)
        
        loss_D = loss_D_real + loss_D_fake
        loss_D.backward()
        optimizer_D.step()
        
        # --- Train Generator + Recovery (Combined Generator) ---
        optimizer_G.zero_grad()
        
        noise = torch.randn(current_bs, seq_len, z_dim).to(device)
        fake_latent = G(noise)
        fake_ecg = R(fake_latent)
        
        outputs = D(fake_ecg)
        loss_G = criterion(outputs, real_labels)  # Try to fool D: label fake as real
        
        loss_G.backward()
        optimizer_G.step()
    
    print(f"Epoch [{epoch}/{num_epochs}], Loss D: {loss_D.item():.4f}, Loss G: {loss_G.item():.4f}")
    
    # Every 5 epochs, plot one real and one fake ECG sample
    if epoch % 1 == 0:
        with torch.no_grad():
            # Use the first sample from the last batch as real ECG for plotting
            real_sample = real_ecg[0].cpu().numpy().squeeze()
            # Generate one fake ECG sample
            noise = torch.randn(1, seq_len, z_dim).to(device)
            fake_sample = R(G(noise)).cpu().numpy().squeeze()
            
        plt.figure(figsize=(12, 4))
        plt.plot(real_sample, label="Real ECG")
        plt.plot(fake_sample, label="Fake ECG", alpha=0.8)
        plt.title(f"Epoch {epoch}")
        plt.legend()
        plt.show()